In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
FILE_BASE_URL_TRAIN_QUESTIONS = "LR1_dev.csv"
FILE_BASE_URL_TRAIN_ANSWERS = "LR1_dev_answers.csv"
FILE_BASE_URL_TEST_QUESTIONS = "LR1.csv"

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.expand_frame_repr', False)

In [4]:
train_df_questions = pd.read_csv(
    FILE_BASE_URL_TRAIN_QUESTIONS
)
train_df_questions.columns = [
    "id",
    "question",
    "options",
    "category"
]

train_df_answers = pd.read_csv(
    FILE_BASE_URL_TRAIN_ANSWERS
)
train_df_answers.columns = [
    "answer_id",
    "answer"
]

train_df = pd.concat(
    (train_df_questions,
    train_df_answers),
    axis = 1
)

train_df = train_df[["id", "question", "options", "category", "answer"]]
train_df.category.value_counts()

category
physics             6
other               5
biology             5
economics           5
chemistry           5
math                4
health              4
psychology          4
history             3
law                 3
computer science    2
engineering         2
business            1
philosophy          1
Name: count, dtype: int64

In [5]:
test_df_questions = pd.read_csv(
    FILE_BASE_URL_TEST_QUESTIONS
)

test_df_questions.columns = [
    "id",
    "question",
    "options",
    "category"
]

test_df_questions.category.value_counts()


category
math                13
physics             11
psychology          10
other                9
law                  8
business             7
biology              7
engineering          7
economics            6
chemistry            6
philosophy           5
health               5
computer science     4
history              2
Name: count, dtype: int64

In [6]:
PROMPTS = {
    "physics": """
Ты эксперт по физике. Реши задачу по физике, используя метод пошагового рассуждения (Chain-of-Thought):

1. Сначала определи, какой раздел физики задействован (механика, электромагнетизм, термодинамика, оптика и т.д.)
2. Выпиши известные величины и что нужно найти
3. Вспомни соответствующие формулы и физические принципы
4. Примени логическое рассуждение по шагам, показывая все вычисления
5. Проверь единицы измерения и размерность
6. Оцени, является ли ответ физически осмысленным

Внимание: После завершения рассуждения дай ТОЛЬКО номер правильного варианта (0, 1, 2 или 3). Не пиши объяснений.
""",

    "chemistry": """
Ты эксперт-химик. Ответь на вопрос по химии, используя методику пошагового анализа:

1. Определи тип химической задачи: стехиометрия, химическая связь, термохимия, кинетика, равновесие и т.д.
2. Если есть химическое уравнение - проверь его баланс
3. Определи молярные массы и стехиометрические соотношения
4. Для вопросов о свойствах - вспомни периодические закономерности
5. Для органической химии - анализируй функциональные группы
6. Проверь соответствие законов сохранения

После анализа дай ТОЛЬКО номер правильного ответа (0-3). Без дополнительных объяснений.
""",

    "biology": """
Ты биолог с глубокими знаниями. Используй метод логической цепочки для ответа на вопрос по биологии:

1. Определи область биологии: молекулярная, клеточная, генетика, эволюция, экология, анатомия
2. Для молекулярных вопросов - вспомни структуры ДНК/РНК, белки, ферменты
3. Для клеточной биологии - рассмотри органеллы и их функции
4. Для генетики - примени законы Менделя, вспомни о ДНК, РНК, мутациях
5. Для эволюции - рассмотри естественный отбор, адаптации
6. Проверь биологическую логику ответа

В конце дай ТОЛЬКО цифру правильного варианта (0, 1, 2 или 3).
""",

    "math": """
Ты математик. Реши математическую задачу, используя строгое логическое рассуждение:

1. Определи тип задачи: алгебра, геометрия, тригонометрия, математический анализ, теория вероятностей
2. Выпиши все данные и неизвестные
3. Вспомни соответствующие теоремы, формулы, свойства
4. Реши задачу шаг за шагом, показывая логические переходы
5. Для геометрии - сделай мысленный чертеж или представь пространственные отношения
6. Проверь решение на наличие ошибок в вычислениях

После решения напиши ТОЛЬКО номер правильного ответа (0-3). Без пояснений.
""",

    "economics": """
Ты экономист-аналитик. Проанализируй экономический вопрос с помощью системного подхода:

1. Определи раздел экономики: микроэкономика, макроэкономика, международная экономика
2. Выдели ключевые экономические концепции (спрос/предложение, издержки, ВВП, инфляция и т.д.)
3. Примени соответствующие экономические модели и теории
4. Проанализируй причинно-следственные связи
5. Рассмотри альтернативные варианты и их последствия
6. Проверь экономическую логику ответа

После анализа выбери и укажи ТОЛЬКО номер верного варианта (0, 1, 2 или 3).
""",

    "psychology": """
Ты профессиональный психолог. Проанализируй психологический вопрос, используя научный подход:

1. Определи область психологии: когнитивная, социальная, клиническая, развивающая, биологическая
2. Вспомни соответствующие психологические теории и исследования
3. Примени принципы научной методологии в психологии
4. Рассмотри различные перспективы и подходы
5. Отдели научно обоснованные факты от популярных представлений
6. Проверь соответствие современным психологическим знаниям

В результате дай ТОЛЬКО цифру правильного ответа (0-3).
""",

    "health": """
Ты специалист в области здравоохранения. Ответь на вопрос о здоровье, используя медицинский подход:

1. Определи тему: анатомия, физиология, питание, общественное здоровье, медицинские процедуры
2. Опирайся на научно доказанные медицинские факты
3. Рассмотри анатомические структуры и физиологические процессы
4. Для вопросов о питании - вспомни питательные вещества и их функции
5. Для медицинских тем - примени принципы доказательной медицины
6. Проверь соответствие современным медицинским знаниям

После анализа напиши ТОЛЬКО номер верного варианта (0, 1, 2 или 3).
""",

    "history": """
Ты историк-аналитик. Проанализируй исторический вопрос, используя критическое мышление:

1. Определи исторический период и регион
2. Вспомни ключевые события, личности, даты
3. Рассмотри причинно-следственные связи исторических процессов
4. Учти различные исторические источники и перспективы
5. Отдели исторические факты от интерпретаций
6. Проверь хронологическую последовательность

В конце укажи ТОЛЬКО цифру правильного ответа (0, 1, 2 или 3).
""",

    "law": """
Ты юрист-аналитик. Проанализируй правовой вопрос, используя юридическое мышление:

1. Определи область права: конституционное, гражданское, уголовное, административное
2. Выдели юридические понятия и принципы
3. Примени соответствующие правовые нормы и прецеденты
4. Проанализируй факты и их юридическую квалификацию
5. Рассмотри все стороны правовой ситуации
6. Проверь логическую последовательность юридических аргументов

После анализа дай ТОЛЬКО номер правильного варианта (0-3).
""",

    "engineering": """
Ты инженер-специалист. Реши инженерную задачу, используя системный подход:

1. Определи инженерную дисциплину: механика, электротехника, строительство и т.д.
2. Выдели технические требования и ограничения
3. Вспомни соответствующие инженерные принципы и формулы
4. Примени метод пошагового проектирования и анализа
5. Проверь техническую осуществимость и безопасность
6. Учти практические аспекты реализации

После решения напиши ТОЛЬКО номер верного ответа (0, 1, 2 или 3).
""",

    "computer science": """
Ты специалист по компьютерным наукам. Реши задачу по информатике, используя алгоритмическое мышление:

1. Определи тему: алгоритмы, структуры данных, программирование, компьютерные сети, базы данных
2. Проанализируй проблему и выдели ключевые элементы
3. Вспомни соответствующие алгоритмы, структуры, принципы
4. Примени логическое рассуждение для решения
5. Для алгоритмических вопросов - пошагово проанализируй выполнение
6. Проверь корректность логических выводов

В результате дай ТОЛЬКО цифру правильного варианта (0-3).
""",

    "business": """
Ты бизнес-эксперт. Проанализируй бизнес-вопрос, используя стратегическое мышление:

1. Определи бизнес-концепцию: менеджмент, маркетинг, финансы, стратегия
2. Выдели ключевые бизнес-принципы и модели
3. Проанализируй ситуацию с точки зрения эффективности и результативности
4. Рассмотри финансовые, маркетинговые и управленческие аспекты
5. Оцени риски и возможности
6. Проверь бизнес-логику предлагаемых решений

После анализа укажи ТОЛЬКО номер правильного ответа (0, 1, 2 или 3).
""",

    "philosophy": """
Ты философ-аналитик. Проанализируй философский вопрос, используя критическое мышление:

1. Определи философскую область: эпистемология, этика, метафизика, логика, философия науки
2. Выдели ключевые философские понятия и теории
3. Примени методы философского анализа и аргументации
4. Рассмотри различные философские позиции и их обоснования
5. Проанализируй логическую структуру аргументов
6. Проверь непротиворечивость и обоснованность выводов

В конце напиши ТОЛЬКО цифру правильного варианта (0, 1, 2 или 3).
""",

    "other": """
Ты эрудит с широкими знаниями. Ответь на вопрос, используя междисциплинарный подход:

1. Проанализируй вопрос и определи его общую тематику
2. Вспомни соответствующие знания из разных областей
3. Примени логическое и критическое мышление
4. Рассмотри вопрос с разных перспектив
5. Проверь факты и логику
6. Исключи маловероятные или неподтвержденные варианты

После анализа дай ТОЛЬКО номер правильного ответа (0, 1, 2 или 3).
"""
}

In [7]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
import os
import gc
import json

/Users/karlkorhonen/проекты/Data_analysis/LLM/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:



def options_parser(
    options
):
    if isinstance(options, list):
        return options
    if isinstance(options, str):
        try:
            return json.loads(options)
        except Exception as e:
            final = []
            if options.startswith('[') and options.endswith(']'):
                options = options.lstrip("[").rstrip("]")
                single_quote = r"'(.*?)'(?=\s*(?:'|$))"
                double_quote = r'"(.*?)"(?=\s*(?:"|$))'
                single = re.findall(single_quote, options, re.DOTALL)
                if not single:
                    single = re.findall(double_quote, options, re.DOTALL)
                for phrase in single:
                    clean = phrase.replace("\\'", "'").replace("\\n", "\n").replace('\\"', '"').strip()
                    final.append(clean)
                return final
            if not final:
                if "," in options:
                    final = list(map(str.strip, options.split(",")))
                    return final
        return options


In [9]:
print(options_parser("['один'\n'два']"))

['один', 'два']
